In [2]:
import pandas as pd

df = pd.read_csv('../data/raw/test.csv')
df.head()

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb
0,137142,CADILUXE,MID-SIZE,3.6,NaN,AS8,X,6.40 km/L,1.77 L/100 km,25.04 MPG (AS)
1,137143,FOLD,VAN - PASSENGER,3.7,6.0,AS6,X,21.11 L/100km,14.38 liters per 100 km,5.53 km per L
2,137144,MITSU,SUV - SMALL,6.0,NaN,AV6,X,9.60 MPG (AS),27.45 mpg Imp.,12.99 MPG (AS)
3,137145,LECUS,MID-SIZE,4.6,NaN,AS8,Z,1.68 L/10km,6.29 km/L,16.42 L/100km
4,137146,BMV,SUV - STANDARD,4.4,8.0,AS8,Z,5.65 MPG (AS),14.75 mpg Imp.,31.52 L/100km


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58776 entries, 0 to 58775
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Id                     58776 non-null  int64 
 1   Make                   58776 non-null  object
 2   Vehicle Class          58243 non-null  object
 3   Engine Size(L)         51537 non-null  object
 4   Cylinders              52822 non-null  object
 5   Transmission           57621 non-null  object
 6   Fuel Type              58006 non-null  object
 7   Fuel Consumption City  57729 non-null  object
 8   Fuel Consumption Hwy   57781 non-null  object
 9   Fuel Consumption Comb  57641 non-null  object
dtypes: int64(1), object(9)
memory usage: 4.5+ MB


In [6]:
df.shape

(58776, 10)